In [99]:
%matplotlib inline
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
from aubio import source, tempo
from numpy import median, diff
song_name = 'A'
mp3_name = 'StepMania/Songs/StepMania 5/{0}/{0}.mp3'.format(song_name)
stepfile_name = 'StepMania/Songs/StepMania 5/{0}/{0}.ssc'.format(song_name)

In [100]:
song_data, sample_rate = librosa.load(mp3_name, sr=22050*4)

In [101]:
tempo, beat_frames = librosa.beat.beat_track(y=song_data, sr=sample_rate)

In [102]:
beat_times = librosa.frames_to_time(beat_frames, sr=sample_rate)

In [105]:
def get_beats_aubio(path):
    #samplerate, win_s, hop_s = 44100, 1024, 512
    samplerate, win_s, hop_s = sample_rate, 1024, 512

    s = source(mp3_name, samplerate, hop_s)
    samplerate = s.samplerate
    o = tempo("specdiff", win_s, hop_s, samplerate)
    # List of beats, in samples
    beats = []
    # Total number of frames read
    total_frames = 0

    while True:
        samples, read = s()
        is_beat = o(samples)
        if is_beat:
            this_beat = o.get_last_s()
            beats.append(this_beat)
            #if o.get_confidence() > .2 and len(beats) > 2.:
            #    break
        total_frames += read
        if read < hop_s:
            break

    # Convert to periods and to bpm 
    if len(beats) > 1:
        if len(beats) < 4:
            print("few beats found in {:s}".format(mp3_name))
        bpms = 60./diff(beats)
        b = median(bpms)
    else:
        b = 0
        print("not enough beats found in {:s}".format(mp3_name))
    return b

def get_beats_old(beat_times):
    range_start = 0
    time_sum = beat_times[1] - beat_times[0]
    beats = 1
    times = []
    for i in range(1, len(beat_times) - 1):
        time_difference = beat_times[i+1] - beat_times[i]

        # check if far from average
        if abs(((time_sum / beats) / time_difference) - 1) > -0.05:
            # more than 5% off average, put it in new time
            bpm_for_section = 60 / (time_sum / beats)
            times.append((range_start, bpm_for_section))
            range_start += beats
            time_sum = 0
            beats = 0

        time_sum += time_difference
        beats += 1
    return times

def get_beats_simple(beat_times):
    buffer = 4
    end_beat = beat_times[len(beat_times) - buffer - 1]
    start_beat = beat_times[buffer]
    beat_total = end_beat - start_beat
    beat = beat_total / (len(beat_times) - 12)
    return [(0, 60./beat)]

def get_beats(beat_times):
    changes = []
    changes_time = []
    for i in range(len(beat_frames) - 1):
        changes.append(beat_frames[i + 1] - beat_frames[i])
        changes_time.append(beat_times[i + 1] - beat_times[i])

    changes.sort()
    changes_time.sort()
    median = changes[int(len(changes) / 2)]

    changes_counted = [abs(change - median) < 2 for change in changes]
    time_changes_counted = list(itertools.compress(changes_time, changes_counted))
    average = sum(time_changes_counted) / len(time_changes_counted)
    [(0, 60./average)]

def get_time_string(beat_times):
    times = get_beats(beat_times)
    time_strings = []
    for time in times:
        if time[0] != 0:
            time_strings.append(',')
        time_strings.append('{:}={:.2f}'.format(time[0], time[1]))
    return ''.join(time_strings)

def test_get_beats(beat_times):
    a = get_beats(beat_times)
    a.append((500, 1))
    time = beat_times[0]
    beat_times_mock = [time]
    for i in range(len(a) - 1):
        for j in range(a[i+1][0] - a[i][0]):
            time += 60/a[i][1]
            beat_times_mock.append(time)

    beat_times_pd = pd.DataFrame(beat_times, columns=['VALUE'])
    beat_times_pd['REAL'] = True
    beat_times_pd
    beat_times_mock_pd = pd.DataFrame(beat_times, columns=['VALUE'])
    beat_times_mock_pd['REAL'] = False
    both = beat_times_pd.append(beat_times_mock_pd)
    both['BEAT']=both.index
    linestyles = ["--", "-"]*390
    fig, (ax) = plt.subplots(1,1, figsize=(20,7))
    graph = sb.pointplot(x='BEAT', y='VALUE', hue='REAL', linestyles=linestyles, data=both, ax=ax)
    
test_get_beats(beat_times)

AttributeError: 'NoneType' object has no attribute 'append'

In [49]:
def write_song_header(output_stepfile):
    keys = ['VERSION', 'TITLE', 'MUSIC', 'OFFSET', 'SAMPLESTART', 'SAMPLELENGTH']
    beat_to_start = 32
    beats_to_play = 32
    header_info = {
        'VERSION': 0.82,
        'TITLE': song_name,
        'MUSIC': '{0}.mp3'.format(song_name),
        'OFFSET': -0.090,
        'SAMPLESTART': beat_times[beat_to_start],
        'SAMPLELENGTH': 60*beats_to_play/tempo
    }
    for key in keys:
        print ("#{0}:{1};".format(key, str(header_info[key])), file=output_stepfile)
        
def write_step_header(output_stepfile):
    print("//---------------dance-single - ----------------", file=output_stepfile)
    keys = ['NOTEDATA', 'CHARTNAME', 'STEPSTYPE', 'DIFFICULTY', 'METER', 'RADARVALUES', 'BPMS']
    step_info = {
        'NOTEDATA': '',
        'CHARTNAME': 'Kommisar',
        'STEPSTYPE': 'dance-single',
        'DIFFICULTY': 'Beginner',
        'METER': 1,
        'RADARVALUES': '0.234,0.292,0.008,0,0,211,212,1,0,0,0,0,0,0,0.234,0.292,0.008,0,0,211,212,1,0,0,0,0,0,0',
        'BPMS': get_time_string(beat_times)
    }
    for key in keys:
        print ("#{0}:{1};".format(key, str(step_info[key])), file=output_stepfile)
        
def write_notes(output_stepfile):
    print ("#NOTES:", file=output_stepfile)
    
    for i in range(80):
        print ("0101\n0001\n0101\n0001\n,", file=output_stepfile)
    print ("0000;", file=output_stepfile)

In [50]:
output_stepfile=open(stepfile_name, 'w')
from __future__ import print_function
write_song_header(output_stepfile)
write_step_header(output_stepfile)
write_notes(output_stepfile)
output_stepfile.close()

In [62]:
get_time_string(beat_times)

'0=89.86'